### Fine tune LLM embeedings model


In [2]:
#!pip install openai==0.28.1
#!pip install openai --upgrade
#!pip install ragas
#!pip install unstructured
#!pip install langchain[all]
#!pip install --upgrade langchain

#!pip install playwright
#!pip install -U selenium unstructured
#!pip install --upgrade langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [1]:
#!pip install pydantic==2.5
#!pip install --upgrade --quiet  langchain_milvus
#!pip install --upgrade scipy

In [1]:
#!pip install -U langchain-openai

In [1]:
#!pip install llama-index-embeddings-langchain

In [1]:
import os, json
import pandas as pd
#import openai
#from langchain.chat_models import ChatOpenAI, ChatGooglePalm
from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] =  OPENAI_API_KEY
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from transformers import (BertTokenizer, 
                          BertModel,
                          BertForMaskedLM, 
                          Trainer, 
                          TrainingArguments, 
                          DataCollatorForLanguageModeling,
                          DataCollatorForWholeWordMask,
                          DataCollatorWithPadding
                         )
from datasets import load_dataset


In [60]:
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import SeleniumURLLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, NLTKTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

ai_model = "gpt-3.5-turbo-0125"
ai_model = "gpt-4o"


llm = ChatOpenAI(model=ai_model, temperature=0.)

In [39]:
from ragas.metrics import (
    context_utilization,
    context_entity_recall,
    context_relevancy,
    answer_relevancy,
    answer_correctness, 
    faithfulness,
    context_precision,
    context_recall,
)

from datasets import Dataset 

from ragas.evaluation import evaluate
import nest_asyncio
#from ragas import evaluate

### Load Documents

In [6]:
def load_url_documets(list_urls, tokens=500):
    
    # Load, chunk and index the contents of the blog.
    loader_url =SeleniumURLLoader( list_urls)
    docs = loader_url.load()
    #text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n","\n"], chunk_size=500, chunk_overlap=50)
    text_splitter = NLTKTextSplitter(chunk_size=tokens, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)
    
    return splits, docs

In [7]:
url_list = ["https://www.nature.com/articles/s41524-023-01062-z",
            "https://www.nature.com/articles/s41699-023-00369-1",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
chunks, documents = load_url_documets(url_list)
print(len(chunks), len(documents))

Created a chunk of size 1063, which is longer than the specified 500
Created a chunk of size 539, which is longer than the specified 500
Created a chunk of size 528, which is longer than the specified 500
Created a chunk of size 577, which is longer than the specified 500
Created a chunk of size 949, which is longer than the specified 500
Created a chunk of size 722, which is longer than the specified 500
Created a chunk of size 1304, which is longer than the specified 500
Created a chunk of size 1227, which is longer than the specified 500
Created a chunk of size 614, which is longer than the specified 500
Created a chunk of size 638, which is longer than the specified 500
Created a chunk of size 2175, which is longer than the specified 500


232 2


### Load Models

In [8]:
# Load a tokenizer specific to BAAI/bge-base-en-v1.5
tokenizer = BertTokenizer.from_pretrained('BAAI/bge-base-en-v1.5')
tokenizer.tokenize_chinese_chars = False

# Load the BAAI/bge-base-en-v1.5 model 
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf_bge = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

model = BertModel.from_pretrained('BAAI/bge-base-en-v1.5')
model.eval() # set the model in evaluation mode

#### LOAD Fine tuning model

# Load the model from the checkpoint directory
model_name ='./results512a/checkpoint-1000'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf_fine_tuned = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

model_fm = BertModel.from_pretrained('./results512a/checkpoint-1000')
model_fm.eval()


/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
No sentence-transformers model found with name ./results512a/checkpoint-1000. Creating a new one with MEAN pooling.
Some weights of BertModel were not initialized from the model checkpoint at ./results512a/checkpoint-1000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at ./results512a/checkpoint-1000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [9]:
import pandas as pd

df_ground_true = pd.read_csv("./data/data_rag/QA_ai4mat_2articles.csv")

questions = df_ground_true['question'].values.tolist()
answers = df_ground_true['answer'].values.tolist()

name = "compare2embedding_model"
_df = pd.read_csv(f"data/data_rag/qa_dict_{name}.csv")

In [10]:
_df.columns

Index(['question', 'answer_original_model', 'answer_FineTuned_model'], dtype='object')

In [40]:
metrics = [
 #  answer_relevancy, 
   faithfulness,
    context_precision,
    context_recall,
    context_entity_recall,
    context_relevancy,
    context_utilization,
]

#### Embedding model BAAI/bge-base-en-v1.5

In [12]:
def get_embeeding(chunks, model, tokenizer):
    """
    Get embeddings for a list of chunks
    """
    
    texts = [t.page_content for t in chunks]
    
    emb = emb_texts(texts, model, tokenizer)

    return emb

def emb_texts(texts, model, tokenizer):   
    
    """
    Get embeddings for a list of texts
    """
    
    model.eval() # set the model in evaluation mode
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Get the embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        # The last hidden state is used as embeddings
        embeddings = outputs.last_hidden_state
        # Pooled output is the mean of the last hidden state
        pooled_emb = outputs.pooler_output.numpy()

    return pooled_emb

In [13]:
emb_texts(["What is DFS?"], model, tokenizer).shape

(1, 768)

### 1. Create MilVus index.

In [14]:
from tqdm import tqdm
from pymilvus import MilvusClient

milvus_client = MilvusClient("./milvus_demo.db")

collection_name = "collection_ai4mat_bge_base"

In [15]:
emb = get_embeeding(chunks, model, tokenizer)

In [16]:
emb.shape

(232, 768)

In [17]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[0]),
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )

In [18]:
data = []

for i, line in enumerate(tqdm(chunks, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[i], "text": chunks[i].page_content, })

milvus_client.insert(collection_name=collection_name, data=data)


Creating embeddings: 100%|█████████████████| 232/232 [00:00<00:00, 72294.10it/s]


{'insert_count': 232,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [19]:

def get_context(
                question, 
                vecstore, 
                collection_name, 
                embedding, 
                top_k=10,
                verbose=False,
               ):
    #print(embedding(q).shape)
    
    search_res = vecstore.search(
         collection_name=collection_name,
         data=[
              embedding(question)
              ],  # Use the `emb_texts` function to convert the question to an embedding vector
         limit=top_k,  # Return top k results
         search_params={"metric_type": "IP", "params": {}},  # Inner product distance
         output_fields=["text"],  # Return the text field
             )

    retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
                                     ]
    if verbose:
        print("Retrieved lines with distances:")
        for line, distance in retrieved_lines_with_distances:
            print(f"- {line}")
            print(f"  Distance: {distance}")
            print()
         
    
    #context = "\n".join(
    #[line for (line, dist) in retrieved_lines_with_distances]
    #      )
    
    context = [line for (line, dist) in retrieved_lines_with_distances]
    
    
    return context

In [20]:
embedding = lambda q: emb_texts([q], model, tokenizer)[0]
collection_name = "collection_ai4mat_bge_base"

In [21]:
contexts = []

for q in questions:
    context = get_context(q[2:], milvus_client, collection_name, embedding, top_k=10)
    contexts.append(context)
    
#context = get_context(q, milvus_client, collection_name, embedding, verbose=verbose)

data_samples = {'question': questions, 
           'ground_truth': answers,
           "contexts": contexts,
          "answer": _df["answer_original_model"].values.tolist()
          }
dataset1 = Dataset.from_dict(data_samples)

In [61]:
nest_asyncio.apply()

#query_engine_bge = build_query_engine(hf_bge, documents)
result1 = evaluate(dataset1, metrics, llm=llm,  raise_exceptions=False, embeddings=hf_bge)

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


In [62]:
print( result1)

{'faithfulness': 0.2450, 'context_precision': 0.0476, 'context_recall': 0.1095, 'context_entity_recall': 0.0088, 'context_relevancy': 0.0548, 'context_utilization': 0.0190}


In [42]:
print( result1)

{'faithfulness': 0.5310, 'context_precision': 0.8262, 'context_recall': 0.5643, 'context_entity_recall': 0.0388, 'context_relevancy': 0.0621, 'context_utilization': 0.8214}


In [23]:
print( result1)

{'faithfulness': 0.5549, 'context_precision': 0.8024, 'context_recall': 0.5429}


In [99]:
print( result1)

{'faithfulness': 0.5704, 'context_precision': 0.7617, 'context_recall': 0.6714}


In [100]:
result1.to_pandas()

,question,ground_truth,contexts,answer,faithfulness,context_precision,context_recall
0,* Which materials are in the dataset?,"The materials in the dataset include MoS2, WSe...",[a The space of defects in 2D materials and b ...,The materials in the dataset include TMDCs (tr...,1.000000,1.000000,1.000000
1,* How many structures are there in the dataset?,"The dataset contains a total of 14,866 structu...",[a The space of defects in 2D materials and b ...,"There are 14,866 structures in the dataset.",0.000000,0.936735,1.000000
2,* How to obtain the dataset?,The datasets analyzed in the study can be obta...,[a The space of defects in 2D materials and b ...,To obtain the dataset of defect configurations...,0.750000,0.988889,1.000000
3,* What is the dataset license?,The dataset is licensed under a Creative Commo...,[Supplementary information\n\nSupplementary Ma...,The dataset license is not specified in the pr...,NaN,0.941518,1.000000
4,* What is the data format?,The data format of the dataset is in CSV forma...,[a The space of defects in 2D materials and b ...,"The data format is structured and organized, c...",0.750000,1.000000,1.000000
5,* How to read the dataset?,"To access and read the dataset, you can visit ...",[a The space of defects in 2D materials and b ...,"To read the dataset, one must first understand...",0.600000,1.000000,1.000000
6,* How to browse the dataset?,The dataset can be browsed and explored by dow...,[a The space of defects in 2D materials and b ...,"To browse the dataset, you can utilize tools s...",1.000000,1.000000,1.000000
7,* Can I work with the dataset without Python?,"Yes, you can work with the dataset without Pyt...",[The same rationale can be used to structure t...,"No, you cannot work with the dataset without P...",NaN,0.906041,1.000000
8,* How to obtain the raw VASP files?,"To obtain the raw VASP files, follow these ste...",[1a).\n\nThere are hundreds of existing 2D mat...,"To obtain the raw VASP files, you can perform ...",0.750000,0.797222,0.000000
9,* What were the settings used for the DFT comp...,The settings used for the DFT computations inc...,[The availability of such high-density defects...,The settings used for the DFT computations wer...,0.333333,0.988889,0.333333


#### Using Fine-tunined BAAI/bge-base-en-v1.5 as embedding model

####  Create MilVus index.

In [24]:
from tqdm import tqdm
from pymilvus import MilvusClient

milvus_client = MilvusClient("./milvus_demo.db")

collection_name = "collection_ai4mat_bge_base_fine_tuned"

In [25]:
emb = get_embeeding(chunks, model_fm, tokenizer)

In [26]:
emb.shape

(232, 768)

In [27]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=len(emb[0]),
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
    )

In [28]:
data = []

for i, line in enumerate(tqdm(chunks, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb[i], "text": chunks[i].page_content, })

milvus_client.insert(collection_name=collection_name, data=data)


Creating embeddings: 100%|████████████████| 232/232 [00:00<00:00, 409200.39it/s]


{'insert_count': 232,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215,

In [29]:
embedding = lambda q: emb_texts([q], model_fm, tokenizer)[0]
collection_name = "collection_ai4mat_bge_base_fine_tuned"

In [30]:
contexts = []

for q in questions:
    context = get_context(q[2:], milvus_client, collection_name, embedding, top_k=10)
    contexts.append(context)
    

data_samples = {'question': questions, 
           'ground_truth': answers,
           "contexts": contexts,
          "answer": _df["answer_FineTuned_model"].values.tolist()
          }
dataset2 = Dataset.from_dict(data_samples)

In [64]:
nest_asyncio.apply()

result2 = evaluate(dataset2, metrics, llm=llm,  raise_exceptions=False, embeddings=hf_fine_tuned)

Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Runner(Thread-34, stopped 140580923373248)>>
Traceback (most recent call last):
  File "/home/oleg/miniconda3/envs/llm/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/home/oleg/miniconda3/envs/llm/lib/python3.10/threading.py", line 1018, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/home/oleg/miniconda3/envs/llm/lib/python3.10/threading.py", line 1336, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/ipykernel/iostream.py", line 559, in flush
    self.pub_thread.schedule(self._flush)
  File "/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/ipykernel/iostream.py", line 251, in schedule
    self._event_pipe.send(b"")
  File "/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/zmq/sugar/soc

Evaluating:   0%|          | 0/210 [00:00<?, ?it/s]

No statements were generated from the answer.


In [65]:
print("Fine-tuned BGE model",result2)
print("Base BGE model",result1)

Fine-tuned BGE model {'faithfulness': 0.5406, 'context_precision': 0.0721, 'context_recall': 0.2238, 'context_entity_recall': 0.0759, 'context_relevancy': 0.0327, 'context_utilization': 0.0606}
Base BGE model {'faithfulness': 0.2450, 'context_precision': 0.0476, 'context_recall': 0.1095, 'context_entity_recall': 0.0088, 'context_relevancy': 0.0548, 'context_utilization': 0.0190}


In [52]:
def print_metrics(result, name=None):
    print(name)
    keys = []
    values = []
    for k,v in result.items():
        print(k,v)
        keys.append(k)
        values.append(v)
    print()
    return keys, values

In [66]:
key, v_m1 = print_metrics(result1, "Base BGE model")
key, v_m2 = print_metrics(result2, "Fine-tuned BGE model")

Base BGE model
faithfulness 0.24500000000000002
context_precision 0.04761904761428571
context_recall 0.10952380952380951
context_entity_recall 0.008843537401198575
context_relevancy 0.05476469632952113
context_utilization 0.019047619045714285

Fine-tuned BGE model
faithfulness 0.5405578898225957
context_precision 0.07207482992798639
context_recall 0.2238095238095238
context_entity_recall 0.07586373907972258
context_relevancy 0.03272856061499737
context_utilization 0.06064625850055782



In [67]:
df_m = pd.DataFrame()
df_m['metric'] = key
df_m["Base BGE model"] = v_m1
df_m["Fine-tuned BGE model"] = v_m2
df_m

,metric,Base BGE model,Fine-tuned BGE model
0,faithfulness,0.245000,0.540558
1,context_precision,0.047619,0.072075
2,context_recall,0.109524,0.223810
3,context_entity_recall,0.008844,0.075864
4,context_relevancy,0.054765,0.032729
5,context_utilization,0.019048,0.060646


In [59]:
df_m = pd.DataFrame()
df_m['metric'] = key
df_m["Base BGE model"] = v_m1
df_m["Fine-tuned BGE model"] = v_m2
df_m

,metric,Base BGE model,Fine-tuned BGE model
0,faithfulness,0.530952,0.578725
1,context_precision,0.826190,0.765728
2,context_recall,0.564286,0.628571
3,context_entity_recall,0.038791,0.102261
4,context_relevancy,0.062099,0.034310
5,context_utilization,0.821429,0.752446


In [32]:
print("Fine-tuned BGE model",result2)
print("Base BGE model",result1)

Fine-tuned BGE model {'faithfulness': 0.5066, 'context_precision': 0.7636, 'context_recall': 0.6452}
Base BGE model {'faithfulness': 0.5549, 'context_precision': 0.8024, 'context_recall': 0.5429}


In [103]:
print("Fine-tuned BGE model",result2)
print("Base BGE model",result1)

Fine-tuned BGE model {'faithfulness': 0.5793, 'context_precision': 0.7432, 'context_recall': 0.6210}
Base BGE model {'faithfulness': 0.5704, 'context_precision': 0.7617, 'context_recall': 0.6714}


In [104]:
result2.to_pandas()

,question,ground_truth,contexts,answer,faithfulness,context_precision,context_recall
0,* Which materials are in the dataset?,"The materials in the dataset include MoS2, WSe...",[a The space of defects in 2D materials and b ...,The materials in the dataset are MoS2 and WSe2.,0.000000,1.000000,1.000000
1,* How many structures are there in the dataset?,"The dataset contains a total of 14,866 structu...",[a The space of defects in 2D materials and b ...,There are a total of 5933 defect configuration...,0.666667,0.988889,1.000000
2,* How to obtain the dataset?,The datasets analyzed in the study can be obta...,[a The space of defects in 2D materials and b ...,The dataset can be obtained by computing datas...,0.666667,0.988889,1.000000
3,* What is the dataset license?,The dataset is licensed under a Creative Commo...,[Supplementary information\n\nSupplementary Ma...,The dataset license is a Creative Commons Attr...,1.000000,0.941518,1.000000
4,* What is the data format?,The data format of the dataset is in CSV forma...,[a The space of defects in 2D materials and b ...,The data format is structured datasets contain...,1.000000,1.000000,1.000000
5,* How to read the dataset?,"To access and read the dataset, you can visit ...",[a The space of defects in 2D materials and b ...,"To read the dataset, one should start by under...",1.000000,0.962654,0.500000
6,* How to browse the dataset?,The dataset can be browsed and explored by dow...,[a The space of defects in 2D materials and b ...,To browse the dataset mentioned in the context...,1.000000,1.000000,1.000000
7,* Can I work with the dataset without Python?,"Yes, you can work with the dataset without Pyt...",[The same rationale can be used to structure t...,"No, you cannot work with the dataset without P...",0.000000,1.000000,1.000000
8,* How to obtain the raw VASP files?,"To obtain the raw VASP files, follow these ste...",[1a).\n\nThere are hundreds of existing 2D mat...,"To obtain the raw VASP files, you can refer to...",0.666667,0.786111,0.000000
9,* What were the settings used for the DFT comp...,The settings used for the DFT computations inc...,[The availability of such high-density defects...,The settings used for the DFT computations wer...,1.000000,0.988889,0.333333


In [106]:
result1.to_pandas().to_csv("./data/data_rag/ragas_metrics_ai4mat_bge_base.csv")

In [107]:
result2.to_pandas().to_csv("./data/data_rag/ragas_metrics_ai4mat_bge_fine_tuned.csv")